# Integrating Agents With Tools

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

In [ ]:
#llama-server -m qwen2.5-coder-7b-instruct-q5_k_m.gguf --port 8080 --jinja
local_gguf_endpoint = "http://localhost:8080/v1/chat/completions"

## Selecting a Playlist for the Party Using `smolagents` and a `ToolCallingAgent`

Let's revisit the previous example where Alfred started party preparations, but this time we'll use a `ToolCallingAgent` to highlight the difference. We'll build an agent that can search the web using DuckDuckGo, just like in our Code Agent example. The only difference is the agent type - the framework handles everything else:

In [ ]:
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool, HfApiModel

agent = ToolCallingAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel(local_gguf_endpoint))

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")


When you examine the agent's trace, instead of seeing `Executing parsed code:`, you'll see something like:

```text
╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': "best music recommendations for a party at Wayne's         │
│ mansion"}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
```  

The agent generates a structured tool call that the system processes to produce the output, rather than directly executing code like a `CodeAgent`.

Now that we understand both agent types, we can choose the right one for our needs. Let's continue exploring `smolagents` to make Alfred's party a success! 🎉

## Tools

In [ ]:
%pip install "smolagents[mcp]"

In [ ]:
import os
from smolagents import ToolCollection, CodeAgent
from mcp import StdioServerParameters
from smolagents import HfApiModel


model = HfApiModel(local_gguf_endpoint)

server_parameters = StdioServerParameters(
    command="uv",
    args=["--quiet", "run", "pubmedmcp"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], model=model, add_base_tools=True)
    agent.run("Please find a remedy for hangover.")